# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [3]:
# your code goes here
import pandas as pd
import numpy as np

def find_col(df, candidates):
    """Return first column name in df.columns that matches any candidate substring (case-insensitive)."""
    cols = list(df.columns)
    for c in candidates:
        for col in cols:
            if c.lower() in col.lower():
                return col
    return None

def safe_print(title, obj):
    print("\n" + "-"*8 + " " + title + " " + "-"*8)
    print(obj)

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

#safe copy and normalize column names for lookups
df_orig = df.copy()
df.columns = [c.strip() for c in df.columns]

#find likely column names 
col_total_claim = find_col(df, ["total_claim", "total_claim_amount", "claim_amount", "total_claims"])
col_clv = find_col(df, ["customer_lifetime_value", "clv", "customer_lifetime"])
col_response = find_col(df, ["response"])
col_policy_type = find_col(df, ["policy_type", "policy type"])
col_gender = find_col(df, ["gender"])
col_monthly_premium = find_col(df, ["monthly_premium", "monthly_premium_auto", "monthly premium"])
col_state = find_col(df, ["state"])
col_effective_date = find_col(df, ["effective_to_date", "effective date", "effective_to"])
col_sales_channel = find_col(df, ["sales_channel", "sales channel", "channel", "saleschannel"])
col_policy_id = find_col(df, ["policy", "policy_id", "policy_id", "policy id", "customer_id"])  # for counting policies

#print detected columns
safe_print("Detected columns", {
    "total_claim": col_total_claim,
    "clv": col_clv,
    "response": col_response,
    "policy_type": col_policy_type,
    "gender": col_gender,
    "monthly_premium": col_monthly_premium,
    "state": col_state,
    "date": col_effective_date,
    "sales_channel": col_sales_channel,
    "policy_identifier": col_policy_id
})

#convert date column and extract month
if col_effective_date:
    df[col_effective_date] = pd.to_datetime(df[col_effective_date], errors="coerce")
    df["month"] = df[col_effective_date].dt.month_name().fillna("Unknown")
    df["month_num"] = df[col_effective_date].dt.month.fillna(0).astype(int)
else:
    #try to parse a column named month or create Unknown
    if "month" in df.columns:
        df["month"] = df["month"].astype(str)
        df["month_num"] = 0
    else:
        df["month"] = "Unknown"
        df["month_num"] = 0

#make sure that numeric columns are numeric
if col_total_claim:
    df[col_total_claim] = pd.to_numeric(df[col_total_claim], errors="coerce")
if col_clv:
    # some CLV values might have % or commas try to clean
    df[col_clv] = df[col_clv].astype(str).str.replace("%","", regex=False).str.replace(",","", regex=False)
    df[col_clv] = pd.to_numeric(df[col_clv], errors="coerce")
if col_monthly_premium:
    df[col_monthly_premium] = pd.to_numeric(df[col_monthly_premium], errors="coerce")

#normalize categorical columns
for c in [col_response, col_policy_type, col_gender, col_state, col_sales_channel]:
    if c:
        df[c] = df[c].astype(str).str.strip()

#(1) 
if col_total_claim and col_response:
    subset1 = df[(df[col_total_claim] < 1000) & (df[col_response].str.lower() == "yes")]
    safe_print("Task 1: Low total_claim_amount (<1000) & responded 'Yes' - sample and count",
               f"Rows: {len(subset1)}\nPreview:\n{subset1.head()}")
else:
    print("\nTask 1: required columns missing (total_claim or response).")

#(2)
if col_response and col_policy_type and col_gender:
    responded_yes = df[df[col_response].str.lower() == "yes"]

    group_cols = [col_policy_type, col_gender]
    agg_cols = {}
    if col_monthly_premium:
        agg_cols[col_monthly_premium] = "mean"
    if col_clv:
        agg_cols[col_clv] = "mean"
    if col_total_claim:
        agg_cols[col_total_claim] = "mean"

    if agg_cols:
        pivot = responded_yes.groupby(group_cols).agg(agg_cols).round(2)
        safe_print("Task 2: Average metrics (responded 'Yes') by policy_type and gender", pivot)
#compare which segments have lower total_claim and higher CLV?
        if col_clv and col_total_claim:
#compute ratio CLV / total_claim to judge profitability
            pivot = pivot.copy()
# guard divide by zero
            pivot["clv_over_claims"] = (pivot[col_clv] / pivot[col_total_claim]).replace([np.inf, -np.inf], np.nan).round(2)
            safe_print("Task 2: add clv_over_claims (higher = better)", pivot.sort_values("clv_over_claims", ascending=False))
    else:
        print("\nTask 2: No numeric columns (monthly_premium or clv or total_claim) found to aggregate.")
else:
    print("\nTask 2: required columns missing (response, policy_type, or gender).")

#(3)
if col_state:
    counts_by_state = df.groupby(col_state).size().rename("num_customers").sort_values(ascending=False)
    safe_print("Task 3: Customers per state (top rows)", counts_by_state.head(20))
    states_gt_500 = counts_by_state[counts_by_state > 500]
    safe_print("Task 3: States with more than 500 customers", states_gt_500)
else:
    print("\nTask 3: state column not found.")

#(4)
col_education = find_col(df, ["education"])
if col_clv and col_education and col_gender:
    stats = df.groupby([col_education, col_gender])[col_clv].agg(["max", "min", "median"]).round(2)
    safe_print("Task 4: CLV stats by education and gender", stats)
    # Some short conclusions
    print("\nTask 4: Conclusions (example):")
    print("Look for education/gender groups with high median CLV (good long-term value).")
    print("Look for groups with low median and high max (high variance) to inspect risk.")
else:
    print("\nTask 4: required columns missing (clv, education, or gender).")

#(5)
if col_state:
    grouped = df.groupby([col_state, "month"]).size().reset_index(name="policies_sold")
    pivot_state_month = grouped.pivot_table(index=col_state, columns="month", values="policies_sold", fill_value=0)
#put months by calendar order if month names exist
    month_order = ["January","February","March","April","May","June","July","August","September","October","November","December"]
#keep only months that appear in columns, but in calendar order
    existing_months = [m for m in month_order if m in pivot_state_month.columns]
    other_months = [c for c in pivot_state_month.columns if c not in existing_months]
    pivot_state_month = pivot_state_month[existing_months + other_months]
    safe_print("Task 5: policies sold by state (rows) x month (columns) - sample", pivot_state_month.head())
else:
    print("\nTask 5: state column not found; cannot compute state x month table.")

#(6)
if col_state:
    total_by_state = df.groupby(col_state).size().rename("total_policies").sort_values(ascending=False)
    top3 = total_by_state.head(3).index.tolist()
    safe_print("Task 6: Top 3 states by total policies", total_by_state.head(10))
#filter grouped table from above for top3
    top3_df = grouped[grouped[col_state].isin(top3)].pivot_table(index=col_state, columns="month", values="policies_sold", fill_value=0)
#put months like before
    existing_months_top3 = [m for m in month_order if m in top3_df.columns]
    other_months_top3 = [c for c in top3_df.columns if c not in existing_months_top3]
    top3_df = top3_df[existing_months_top3 + other_months_top3]
    safe_print("Task 6: policies sold by month for top 3 states", top3_df)
else:
    print("\nTask 6: state column not found; cannot compute top 3 states per month.")

#(7)
if col_sales_channel and col_response:
    channel_counts = df.groupby(col_sales_channel)[col_response].value_counts().unstack(fill_value=0)
    if "Yes" in channel_counts.columns or "yes" in channel_counts.columns:
        yes_col = "Yes" if "Yes" in channel_counts.columns else "yes"
        channel_counts["response_rate"] = (channel_counts[yes_col] / channel_counts.sum(axis=1)).round(3)
    else:
        channel_counts["response_rate"] = (channel_counts.filter(regex='(?i)^yes$').sum(axis=1) / channel_counts.sum(axis=1)).round(3)
    safe_print("Task 7: Response rate by sales channel", channel_counts[["response_rate"]] if "response_rate" in channel_counts.columns else channel_counts)
else:
    channel_cols = [c for c in df.columns if any(k in c.lower() for k in ["channel","branch","web","mail","call","online"])]
    channel_cols = [c for c in channel_cols if c not in [col_sales_channel, col_state, col_policy_type, col_gender, col_response]]
    if channel_cols and col_response:
        melted = df.melt(id_vars=[col_response], value_vars=channel_cols, var_name="channel", value_name="channel_value")
        melted["channel_active"] = melted["channel_value"].astype(str).str.lower().isin(["1","true","yes","y"])
        channel_stats = melted.groupby("channel").apply(
            lambda g: pd.Series({
                "n_customers": g.shape[0],
                "n_responded_yes": g.loc[g[col_response].str.lower()=="yes", "channel_active"].sum(),
                "response_rate": (g.loc[g[col_response].str.lower()=="yes", "channel_active"].sum() / g["channel_active"].sum()) if g["channel_active"].sum()>0 else np.nan
            })
        ).sort_values("response_rate", ascending=False)
        safe_print("Task 7 (melt path): channel response stats", channel_stats)
    else:
        print("\nTask 7: Could not detect sales channel structure. Please check column names for sales channel info.")

print("\nDone: all tasks attempted.")



-------- Detected columns --------
{'total_claim': None, 'clv': None, 'response': 'Response', 'policy_type': 'Policy Type', 'gender': 'Gender', 'monthly_premium': 'Monthly Premium Auto', 'state': 'State', 'date': None, 'sales_channel': 'Sales Channel', 'policy_identifier': 'Months Since Policy Inception'}

Task 1: required columns missing (total_claim or response).

-------- Task 2: Average metrics (responded 'Yes') by policy_type and gender --------
                       Monthly Premium Auto
Policy Type    Gender                      
Corporate Auto F                      94.30
               M                      92.19
Personal Auto  F                      99.00
               M                      91.09
Special Auto   F                      92.31
               M                      86.34

-------- Task 3: Customers per state (top rows) --------
State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
nan            631
Name: num_cust